# LLM OpenAI Cloud

# Minimalbeispiel: OpenAI LLM aus Python

**Schritte**:
1. API Key besorgen (OpenAI Dashboard) und als Umgebungsvariable setzen.
   - PowerShell (nur aktuelle Sitzung): `$env:OPENAI_API_KEY = "sk-..."`
   - Dauerhaft (Benutzer): `[Environment]::SetEnvironmentVariable("OPENAI_API_KEY","sk-...","User")`
2. Paket `openai` installieren (siehe nächste Zelle).
3. Code-Zelle ausführen, Antwort erscheint unter der Zelle.

**Hinweis**: Der Key darf nicht öffentlich geteilt werden. Wenn keine Umgebungsvariable gesetzt ist, fragt der Code interaktiv nach dem Key.

**Modelle**: Für ein günstiges Beispiel wird `gpt-4.1-mini` verwendet (anpassbar).


In [5]:
# Installation des OpenAI Python Pakets
!python -m pip install --quiet --upgrade openai



[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import os, getpass
from openai import OpenAI

# API Key laden oder abfragen
api_key = os.getenv("OPENAI_API_KEY") or getpass.getpass("OpenAI API Key eingeben: ")
client = OpenAI(api_key=api_key)

prompt = "Schreibe einen einzigen kurzen Satz über Computer Vision auf Deutsch."
# Neues Responses-API (empfohlen)
resp = client.responses.create(
    model="gpt-4.1-mini",
    input=prompt,
)

# Ausgabe robuster extrahieren (Responses oder ältere Chat-Struktur)
text = None
try:
    # Neues Schema
    text = resp.output[0].content[0].text
except Exception:
    try:
        # Fallback auf Chat Completion Struktur
        text = resp.choices[0].message.content
    except Exception as e:
        text = f"Konnte Text nicht extrahieren: {e}"
print(text)


# Interaktiver Chat mit Verlauf (Responses API)

- Führe diese Zelle und danach die nächste Code-Zelle aus.
- In der Chat-Zelle kannst du Nachrichten eingeben; mit `exit`/`quit` beendest du.
- API-Key wird aus `OPENAI_API_KEY`, aus der Datei `openai_api_key.txt` (im aktuellen Ordner) oder interaktiv abgefragt.
- Modell ist per Variable anpassbar (Standard: `gpt-4.1-mini`).


In [ ]:
import os, json, getpass
from openai import OpenAI

# OpenAI Client erstellen (ENV > Datei > Eingabe)
def get_openai_client():
    key = os.getenv("OPENAI_API_KEY")
    if not key and os.path.exists("openai_api_key.txt"):
        with open("openai_api_key.txt", "r", encoding="utf-8") as f:
            key = f.read().strip()
    if not key:
        key = getpass.getpass("OpenAI API Key eingeben: ")
    return OpenAI(api_key=key)

client = get_openai_client()
model = "gpt-4.1-mini"  # anpassbar, z.B. "gpt-4o-mini"

history = []  # Verlauf im Chat-Format: [{role, content}, ...]
print("Chat gestartet. Tippe 'exit' zum Beenden.\n")

while True:
    try:
        user_input = input("Du> ").strip()
    except EOFError:
        break
    if not user_input:
        continue
    if user_input.lower() in ("exit", "quit", "q"):
        break

    history.append({"role": "user", "content": user_input})

    try:
        # Responses API mit Chat-Verlauf
        response = client.responses.create(
            model=model,
            input=history,
        )
        try:
            bot_text = response.output_text  # praktischer Shortcut, falls verfügbar
        except Exception:
            # Fallback auf strukturierte Felder
            try:
                bot_text = response.output[0].content[0].text
            except Exception as e:
                bot_text = f"Konnte Text nicht extrahieren: {e}"
    except Exception as e:
        bot_text = f"API-Fehler: {e}"

    history.append({"role": "assistant", "content": bot_text})

    print(f"Bot> {bot_text}\n")

print("\nVerlauf (JSON):")
print("-"*60)
print(json.dumps(history, ensure_ascii=False, indent=2))
